In [298]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)


In [299]:
df = pd.read_csv('data_cleaned.csv')

In [300]:
columns = ['KD',
       'SUB.ATT', 'REV.', 'CTRL', 'ATTEMPTED SIG.STR.', 'LANDED SIG.STR.',
       'ATTEMPTED TOTAL STR.', 'LANDED TOTAL STR.', 'ATTEMPTED TD',
       'LANDED TD', 'ATTEMPTED HEAD', 'LANDED HEAD', 'ATTEMPTED BODY',
       'LANDED BODY', 'ATTEMPTED LEG', 'LANDED LEG', 'ATTEMPTED DISTANCE',
       'LANDED DISTANCE', 'ATTEMPTED CLINCH', 'LANDED CLINCH',
       'ATTEMPTED GROUND', 'LANDED GROUND', 'KDopp', 'SUB.ATTopp',
       'REV.opp', 'CTRLopp', 'ATTEMPTED SIG.STR.opp', 'LANDED SIG.STR.opp',
       'ATTEMPTED TOTAL STR.opp', 'LANDED TOTAL STR.opp', 'ATTEMPTED TDopp',
       'LANDED TDopp', 'ATTEMPTED HEADopp', 'LANDED HEADopp',
       'ATTEMPTED BODYopp', 'LANDED BODYopp', 'ATTEMPTED LEGopp',
       'LANDED LEGopp', 'ATTEMPTED DISTANCEopp', 'LANDED DISTANCEopp',
       'ATTEMPTED CLINCHopp', 'LANDED CLINCHopp', 'ATTEMPTED GROUNDopp',
       'LANDED GROUNDopp', 'SIG.STR. %','TD %','SIG.STR. %opp','TD %opp']

In [301]:
notcols = [i for i in df.columns if i not in columns]

# FUNCTIONS

In [302]:
def minute_to_second(time):
    '''
    INPUT: str? (12:32)
    OUTPUT: int (752)
    
    '''
    if ':' in str(time):
        time = str(time)
        time_splitted = time.split(':')
        result = int(time_splitted[0])*60 + int(time_splitted[1])
    else:
        result = np.nan

    return result

In [303]:
df['OUTCOME'] = df['OUTCOME'][(df['OUTCOME']!= 'D/D') & (df['OUTCOME'] != 'NC/NC')]

In [304]:
dfmerged =df

In [305]:
dfletsgo = dfmerged[notcols].drop(['ROUND', 'DETAILS'], axis = 1)

In [306]:
dfmean = dfmerged.groupby(['EVENT', 'BOUT', 'FIGHTER'])[columns].mean().reset_index()

In [307]:
for column in columns:
    dfmean = dfmean.rename(columns = {column : column + ' mean'} )

In [308]:
dfsumrate = dfmerged.groupby(['EVENT', 'BOUT', 'FIGHTER'])[columns].sum().reset_index()

In [309]:
columnslist = ['SIG.STR.','TOTAL STR.', 'TD', 'HEAD', 'BODY','LEG', 'DISTANCE', 'CLINCH', 'GROUND']
for strike in columnslist:
    dfsumrate['AVG ' + strike ] = dfsumrate['ATTEMPTED ' + strike]/ dfsumrate['LANDED ' + strike]
    dfsumrate['AVG ' + strike + 'opp'] = dfsumrate['ATTEMPTED ' + strike + 'opp']/ dfsumrate['LANDED ' + strike +'opp'] 

In [310]:
for column in columns:
    dfsumrate = dfsumrate.rename(columns = {column : column + ' sum'} )

In [311]:
dfsumrate.replace([np.inf, -np.inf], 0, inplace=True)

In [312]:
dfnewstats = pd.merge(dfsumrate, dfmerged, on= ['EVENT', 'BOUT', 'FIGHTER'])

In [313]:
dffull = pd.merge(dfsumrate, dfmean, on = ['EVENT', 'BOUT', 'FIGHTER'])

In [314]:
dfletsgo = dfletsgo.drop_duplicates()

In [315]:
dfnewstats = pd.merge(dffull, dfletsgo, on = ['EVENT', 'BOUT', 'FIGHTER'])

In [316]:
dfnewstats['OUTCOME'] = dfnewstats['OUTCOME'][(dfnewstats['OUTCOME']!= 'D/D') & (dfnewstats['OUTCOME'] != 'NC/NC')]

In [317]:
from datetime import datetime
dfnewstats['DOB'] = pd.to_datetime(dfnewstats['DOB'], format='%b %d, %Y')
dfnewstats['DOBopp'] = pd.to_datetime(dfnewstats['DOBopp'], format='%b %d, %Y')
dfnewstats['DATE'] = pd.to_datetime(dfnewstats['DATE'], format='%B %d, %Y')

In [318]:
dfnewstats['DOB'] = dfnewstats['DOB'].astype(object).where(df['DOB'].notnull(), None)
dfnewstats['DOB'] = dfnewstats['DOB'].apply(lambda x: x if pd.NaT else int(x.utcnow().timestamp()))

In [319]:
dfnewstats['DOBopp'] = dfnewstats['DOBopp'].astype(object).where(df['DOBopp'].notnull(), None)
dfnewstats['DOBopp'] = dfnewstats['DOBopp'].apply(lambda x: x if pd.NaT else int(x.utcnow().timestamp()))

In [320]:
dfnewstats['DATE'] = dfnewstats['DATE'].astype(object).where(df['DATE'].notnull(), None)
dfnewstats['DATE'] = dfnewstats['DATE'].apply(lambda x: x if pd.NaT else int(x.utcnow().timestamp()))

In [321]:
dftesting = dfnewstats

In [322]:
cols = list(dfnewstats.columns)
notcol =['EVENT',
 'BOUT',
 'ROUND',
 'FIGHTER', 'OUTCOME',
 'WEIGHTCLASS',
 'METHOD',
 'WIN ROUND',
 'TIME',
 'TIME FORMAT',
 'REFEREE',
 'DETAILS',
 'Fight Result URL',
 'Fight Event URL',
 'DATE',
 'LOCATION',
 'Fighter Stats URL',
 'HEIGHT',
 'WEIGHT',
 'REACH',
 'STANCE',
 'DOB',
 'OPPONENT', 'Fighter Stats URLopp',
 'HEIGHTopp',
 'WEIGHTopp',
 'REACHopp',
 'STANCEopp',
 'DOBopp']
res = [i for i in cols if i not in notcol]

In [323]:
for column in res:
    dftesting[column + 'rolling mean'] = dftesting.groupby('Fighter Stats URL' )[column].apply(lambda x: x.rolling(3, min_periods = 3).mean())
    dftesting[column + 'percent change'] = dftesting.groupby('Fighter Stats URL')[column].apply(lambda x: x.pct_change())

/tmp/ipykernel_8102/1935856063.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[column + 'rolling mean'] = dftesting.groupby('Fighter Stats URL' )[column].apply(lambda x: x.rolling(3, min_periods = 3).mean())
/tmp/ipykernel_8102/1935856063.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys

/tmp/ipykernel_8102/1935856063.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[column + 'percent change'] = dftesting.groupby('Fighter Stats URL')[column].apply(lambda x: x.pct_change())
/tmp/ipykernel_8102/1935856063.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[co

/tmp/ipykernel_8102/1935856063.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[column + 'percent change'] = dftesting.groupby('Fighter Stats URL')[column].apply(lambda x: x.pct_change())
/tmp/ipykernel_8102/1935856063.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[co

/tmp/ipykernel_8102/1935856063.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[column + 'rolling mean'] = dftesting.groupby('Fighter Stats URL' )[column].apply(lambda x: x.rolling(3, min_periods = 3).mean())
/tmp/ipykernel_8102/1935856063.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys

/tmp/ipykernel_8102/1935856063.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[column + 'percent change'] = dftesting.groupby('Fighter Stats URL')[column].apply(lambda x: x.pct_change())
/tmp/ipykernel_8102/1935856063.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[co

...

/tmp/ipykernel_8102/1935856063.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dftesting[column + 'percent change'] = dftesting.groupby('Fighter Stats URL')[column].apply(lambda x: x.pct_change())
/tmp/ipykernel_8102/1935856063.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[column + 'rolling mean'] = dftesting.groupby('Fighter Stats URL' )[column].apply(lambda x: x.rolling(3, min_periods = 3).

/tmp/ipykernel_8102/1935856063.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dftesting[column + 'rolling mean'] = dftesting.groupby('Fighter Stats URL' )[column].apply(lambda x: x.rolling(3, min_periods = 3).mean())
/tmp/ipykernel_8102/1935856063.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[column + 'percent change'] = dftesting.groupby('Fighter Stats URL')[column].apply(lambda x: x.pct_ch

/tmp/ipykernel_8102/1935856063.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dftesting[column + 'percent change'] = dftesting.groupby('Fighter Stats URL')[column].apply(lambda x: x.pct_change())
/tmp/ipykernel_8102/1935856063.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[column + 'rolling mean'] = dftesting.groupby('Fighter Stats URL' )[column].apply(lambda x: x.rolling(3, min_periods = 3).

/tmp/ipykernel_8102/1935856063.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dftesting[column + 'rolling mean'] = dftesting.groupby('Fighter Stats URL' )[column].apply(lambda x: x.rolling(3, min_periods = 3).mean())
/tmp/ipykernel_8102/1935856063.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[column + 'percent change'] = dftesting.groupby('Fighter Stats URL')[column].apply(lambda x: x.pct_ch

/tmp/ipykernel_8102/1935856063.py:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting[column + 'percent change'] = dftesting.groupby('Fighter Stats URL')[column].apply(lambda x: x.pct_change())
/tmp/ipykernel_8102/1935856063.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dftesting[column + 'percent change'] = dftesting.groupby('Fighter Stats URL')[column].apply(lambda x: x.pct_change())
/tmp/ipykerne

In [324]:
dftesting['one'] = 1

/tmp/ipykernel_8102/1147337539.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dftesting['one'] = 1


In [325]:
dftesting['fight history'] = dftesting.groupby('Fighter Stats URL')['one'].apply(lambda x: x.cumsum())

/tmp/ipykernel_8102/4231710358.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  dftesting['fight history'] = dftesting.groupby('Fighter Stats URL')['one'].apply(lambda x: x.cumsum())
/tmp/ipykernel_8102/4231710358.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dftesting['fight history'] = dftesting.groupby('Fighter Stats URL')['one'].apply(lambda x: x.cumsum())


In [326]:
dftesting[['EVENT', 'BOUT', 'FIGHTER','Fighter Stats URL', 'WEIGHTCLASS', 'METHOD','TIME FORMAT', 'REFEREE',
       'Fight Result URL', 'OPPONENT', 'STANCEopp', 'STANCE','Fighter Stats URLopp', 'LOCATION', 'OUTCOME']] = dftesting[['EVENT', 'BOUT', 'FIGHTER','Fighter Stats URL', 'WEIGHTCLASS', 'METHOD','TIME FORMAT', 'REFEREE',
       'Fight Result URL', 'OPPONENT', 'STANCEopp', 'STANCE','Fighter Stats URLopp', 'LOCATION',  'OUTCOME']].astype('category')

In [327]:
cat_columns = dftesting.select_dtypes(['category']).columns

In [328]:
dftesting[cat_columns] = dftesting[cat_columns].apply(lambda x: x.cat.codes)

In [329]:
dftesting['DOB'] = pd.to_numeric(pd.to_datetime(dftesting['DOB']))
dftesting['DOBopp'] = pd.to_numeric(pd.to_datetime(dftesting['DOBopp']))
dftesting['DATE'] = pd.to_numeric(pd.to_datetime(dftesting['DATE']))

In [330]:
dftesting['AGEdiff'] = dftesting['DOB'] - dftesting['DOBopp']

In [331]:
dftesting['relativeyouth'] = dftesting['DATE'] - dftesting['DOB']
dftesting['reachdiff'] = dftesting['REACH'] - dftesting['REACHopp']

In [332]:
dftesting = dftesting[dftesting['OUTCOME'] != -1]

In [333]:
dftesting = dftesting.drop('one', axis = 1)

In [349]:
dfshift = dftesting.groupby(['DATE', 'EVENT', 'BOUT', 'FIGHTER']).shift(1)
dfshift = dfshift.assign(date=dftesting['DATE'], event = dftesting['EVENT'], bout = dftesting['BOUT'],fighter =  dftesting['FIGHTER'])
dfshift = dfshift.reset_index()
dfshift = dfshift.replace(np.inf, np.nan)

In [353]:
from sklearn.model_selection import TimeSeriesSplit
from tpot import TPOTClassifier
from dask.distributed import Client
import dask.dataframe as dd
import dask.multiprocessing
from sklearn.metrics import accuracy_score
tss = TimeSeriesSplit(n_splits = 3)
df  = dfshift
X = df.drop(labels=['Fight Event URL', 'OUTCOME'], axis=1)
y = dftesting['OUTCOME'] 
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [356]:
import pandas as pd
from sklearn import preprocessing
import pandas as pd
from sklearn import preprocessing
x = X_train.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X_train=pd.DataFrame(x_scaled, columns=X_train.columns)

In [357]:
pipeline_optimizer = TPOTClassifier(use_dask=True, warm_start=True, config_dict="TPOT cuML", verbosity =3)

In [358]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import dask_cudf
import cudf
cluster = LocalCUDACluster()
client = Client(cluster)
client


/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/dask_cuda/utils.py:258: UserWarning: Cannot get CPU affinity for device with index 0, setting default affinity
  warnings.warn(
2023-01-26 13:34:53,558 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-01-26 13:34:53,558 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 31.30 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36421,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 31.30 GiB
Comm: tcp://127.0.0.1:36669,Total threads: 1
Dashboard: http://127.0.0.1:34169/status,Memory: 31.30 GiB
Nanny: tcp://127.0.0.1:45097,


In [359]:
y_train.value_counts()

1    6966
0    3228
Name: OUTCOME, dtype: int64

In [360]:
pipeline_optimizer.fit(X_train, y_train)

19 operators have been imported by TPOT.
Imputing missing values in feature set
[W] [13:35:16.583662] QWL-QN: max iterations reached
[W] [13:35:16.584394] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [13:35:23.321129] QWL-QN: max iterations reached
[W] [13:35:23.321907] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [13:35:25.355627] QWL-QN: max iterations reached
[W] [13:35:25.356185] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [13:35:34.282444] QWL-QN: max iterations reached
[W] [13:35:34.283071] Maximum iterations reached before solver is converged. To increase model accuracy you can

Optimization Progress:   0%|          | 0/10100 [00:00<?, ?pipeline/s]

/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used

[W] [13:35:54.091732] QWL-QN: max iterations reached
[W] [13:35:54.091785] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [13:35:55.712411] QWL-QN: max iterations reached
[W] [13:35:55.712447] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [13:35:57.271649] QWL-QN: max iterations reached
[W] [13:35:57.271684] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the input data.
[W] [13:35:58.872333] QWL-QN: max iterations reached
[W] [13:35:58.872373] Maximum iterations reached before solver is converged. To increase model accuracy you can increase the number of iterations (max_iter) or improve the scaling of the inpu

/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used

...

Skipped pipeline #10018 due to time out. Continuing to the next pipeline.


2023-01-28 09:41:42,641 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-01-28 09:41:44,183 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-01-28 09:41:46,050 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-01-28 09:42:02,560 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-01-28 09:42:11,568 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-01-28 09:42:16,524 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2023-01-28 09:42:17,890 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2023-01-28 09:42:19,568 - distributed.utils_perf - WARNING - full garbage collections took

/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/cluster/_agglomerative.py:983: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(
2023-01-28 10:52:44,802 - distributed.ut


Generation 100 - Current Pareto front scores:

-1	0.6834412951373631	XGBClassifier(CombineDFs(input_matrix, input_matrix), XGBClassifier__alpha=10, XGBClassifier__learning_rate=0.01, XGBClassifier__max_depth=3, XGBClassifier__min_child_weight=16, XGBClassifier__n_estimators=100, XGBClassifier__n_jobs=1, XGBClassifier__subsample=0.05, XGBClassifier__tree_method=gpu_hist, XGBClassifier__verbosity=0)

-2	0.6837357014180304	XGBClassifier(CombineDFs(RBFSampler(CombineDFs(input_matrix, input_matrix), RBFSampler__gamma=0.15000000000000002), CombineDFs(input_matrix, input_matrix)), XGBClassifier__alpha=10, XGBClassifier__learning_rate=0.01, XGBClassifier__max_depth=3, XGBClassifier__min_child_weight=16, XGBClassifier__n_estimators=100, XGBClassifier__n_jobs=1, XGBClassifier__subsample=0.05, XGBClassifier__tree_method=gpu_hist, XGBClassifier__verbosity=0)


TPOTClassifier(config_dict='TPOT cuML', use_dask=True, verbosity=3,
               warm_start=True)

In [361]:
print(pipeline_optimizer.score(X_test, y_test))

Imputing missing values in feature set
0.5974102413184226


/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/metrics/_scorer.py:794: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(


In [363]:
import pandas as pd
from sklearn import preprocessing
x = X_test.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X_test=pd.DataFrame(x_scaled, columns=X_test.columns)

In [364]:
print(pipeline_optimizer.score(X_test, y_test))

Imputing missing values in feature set
0.5974102413184226


/home/jaleu/miniconda3/envs/rapids-22.12/lib/python3.9/site-packages/sklearn/metrics/_scorer.py:794: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(


In [362]:
pipeline_optimizer.export('tpot_digits_pipeline5.py')